In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# default_exp read_data
# all_data

In [ ]:
# export
from pathlib import Path
import pandas as pd

# Read data

## Get file paths

### by year

In [ ]:
COMMENTS_DIR = '../data/comments/by_date/'

In [ ]:
YEAR = 2019

In [ ]:
# export
def get_comments_paths_year(COMMENTS_DIR, YEAR):
    comments_dir_path = Path(COMMENTS_DIR)
    comments_paths = list(comments_dir_path.glob(f'{YEAR}*.csv'))
    return comments_paths    

In [ ]:
get_comments_paths_year(COMMENTS_DIR, '2019')

[Path('../data/comments/by_date/2019-05-07_21:11:36___2019-05-07_21:59:59.csv'),
 Path('../data/comments/by_date/2019-08-07_21:12:15___2019-08-07_21:59:59.csv'),
 Path('../data/comments/by_date/2019-07-14_21:06:51___2019-07-14_21:59:59.csv'),
 Path('../data/comments/by_date/2019-07-01_21:59:59___2019-07-01_21:19:55.csv'),
 Path('../data/comments/by_date/2019-05-14_21:15:37___2019-05-14_21:59:59.csv'),
 Path('../data/comments/by_date/2019-06-07_21:17:11___2019-06-07_21:59:59.csv'),
 Path('../data/comments/by_date/2019-02-01_22:59:59___2019-02-01_22:02:38.csv'),
 Path('../data/comments/by_date/2019-02-07_22:06:26___2019-02-07_22:59:59.csv'),
 Path('../data/comments/by_date/2019-06-01_21:59:59___2019-06-01_21:09:37.csv'),
 Path('../data/comments/by_date/2019-11-07_22:24:23___2019-11-07_22:59:59.csv'),
 Path('../data/comments/by_date/2019-03-14_22:06:05___2019-03-14_22:59:59.csv'),
 Path('../data/comments/by_date/2019-10-07_21:17:33___2019-10-07_21:59:59.csv'),
 Path('../data/comments/by_d

In [ ]:
assert len(get_comments_paths_year(COMMENTS_DIR, '2019')) == 48

In [ ]:
assert len(get_comments_paths_year(COMMENTS_DIR, '2020')) == 48

In [ ]:
comment_paths_year = get_comments_paths_year(COMMENTS_DIR, YEAR)

### by subreddit

In [ ]:
COMMENTS_DIR_SUBR = '../data/comments/subr/'

In [ ]:
SUBR = 'conspiracy'

In [ ]:
# export
def get_comments_paths_subr(COMMENTS_DIR_SUBR, SUBR):
	comments_subr_dir_path = Path(COMMENTS_DIR_SUBR)
	comments_subr_paths = list(comments_subr_dir_path.glob(f'{SUBR}*.csv'))
	return comments_subr_paths

In [ ]:
comments_paths_subr = get_comments_paths_subr(COMMENTS_DIR_SUBR, SUBR)

## Read comments

### Read `1` comments `csv` file

In [ ]:
fpath = comment_paths_year[0]

In [ ]:
# export
def read_comm_csv(fpath):
    try:
        # removed because new method for writing retrieved data out already does date conversion beforehand
        # date_parser = lambda x: pd.to_datetime(x, unit='s', errors='coerce')
        comments = pd.read_csv(
            fpath,
            usecols=['id', 'created_utc', 'author', 'subreddit', 'body'],
            dtype={
                'id': 'string',
                # 'created_utc': int, s. above
                'author': 'string',
                'subreddit': 'string',
                'body': 'string'
            },
            parse_dates=['created_utc'],
            # date_parser=date_parser,
            low_memory=False,
            lineterminator='\n'
        )
        comments_clean = comments\
            .dropna()\
            .drop_duplicates(subset='id')
        return comments_clean
    except FileNotFoundError:
        print(f'{fpath} not found on disk')
    except pd.errors.EmptyDataError:
        print(f'{fpath} is empty')

In [ ]:
comments = read_comm_csv(fpath)

In [ ]:
comments.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200000 entries, 0 to 199999
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   author       200000 non-null  string        
 1   body         200000 non-null  string        
 2   created_utc  200000 non-null  datetime64[ns]
 3   id           200000 non-null  string        
 4   subreddit    200000 non-null  string        
dtypes: datetime64[ns](1), string(4)
memory usage: 9.2 MB


### Read multiple comment `csv` files

In [ ]:
# export
def read_comm_csvs(fpaths: list):
    comments_lst = []
    for fpath in fpaths:
        comments = read_comm_csv(fpath)
        comments_lst.append(comments)
    comments_concat = pd.concat(
        comments_lst,
        axis=0,
        ignore_index=True
    )
    return comments_concat

In [ ]:
comments = read_comm_csvs(comment_paths_year)

In [ ]:
comments.value_counts('subreddit')

subreddit
AskReddit             429516
politics              146023
memes                  99027
teenagers              89685
dankmemes              84107
                       ...  
no_u                       1
CuteBobby                  1
no_drama                   1
WorldBoxGodSandbox         1
FatFurryPorn               1
Length: 66885, dtype: int64

## Inspect comments

In [ ]:
COMMENTS_DIR = '../data/comments/by_date/'

In [ ]:
YEAR = 2019

In [ ]:
fpaths = get_comments_paths_year(COMMENTS_DIR, '2019')

In [ ]:
comments = read_comm_csvs(fpaths)

In [ ]:
pd.set_option('display.max_rows', 500)
pd.options.display.max_colwidth = 200


hits = (comments
	.filter(['body'])
	.query('body.str.contains("spreader")')
	.head(100)
)

hits.style.set_properties(**{'text-align': 'left'})
hits

,body
233362,"No, record two separate takes of the same vocals. Copying and pasting vocals to two tracks will only make it louder. Recording two separate takes and making one lower in volume will give the ‘Doub..."
316629,"I am not familiar with that brand, but I own a couple of pieces of bare anodized aluminum cookware. I would not recommend them. Bare anodized aluminum is easily the most stick-prone material I ha..."
382123,And here I was going to ask if it fitted into your garage or did you have to take the salt spreader off ...🤣
423532,"Yeah as I said, I assume that the temperature readings may be higher then I expected because of the way the dies are connected to the heatspreader. I think der8auer, the German overclocker, [concl..."
456937,"They definitely don’t “age”—I had one running for my entire Interactions run and it worked like a champ the entire time. When you say they aren’t producing much mana, what do you mean? Do they fi..."
626784,I have yet to see a monohull without a backstay. The swept back spreader puts a slight bend in the mast which helps in performance. Downwind does suck which is why you use a spinnaker instead.
761122,"Even better, that's actually Bodvarr Ribspreader"
999738,That is in this case part of the chastity belt. But based on speculation I think it might just be called a 'butt spreader' or 'cheek spreader'
1228940,"But imagine cooling something with a heatspreader/chip size of Threadripper. When you have more surface area, you can get bigger/beefier coolers and you can use something like the Silver Arrow for..."
1281884,System agent chip is under palmrest and covered by the heat spreader on the left side so likely its that.
